# Pick and Place Example

In this example, we will use the feature library to recognize, pick and then place objects set on a table below the hand. 
We assume the hand to be placed approximately 20cm above the objects of interest (1). If everything goes right, you should expect a point cloud as shown in (2), a segmentation result as in (3) and a resulting grasp as shown in (4).

<img src="pick_and_place.png">

In [4]:
#Imports
import rmlib
rm = rmlib.RMLib()

Setting up robot please wait...
Robot Ready!


# Load Feature Library

A <i>feature</i> is a datastructure that describes the sequence of image processing steps (`capture_process_list`) and their parameters that the SmartHand needs to perform to find a specific object. Features can be created with Robotic Materials' feature editor to differentiate arbitrary objects, or a "default feature" can be used that works with most unknown objects.

In [5]:
#Load in the feature library to use default feature or features created in the feature editor (coming soon).
lib = rm.load_feature_lib()

# Use print_feature_lib() to print a readable version of the feature lib
rm.print_feature_lib(lib)

Library Features:
default:
       capture_process_list :
              0 :
              descriptor  =  downsample
              leaf_size  =  0.003
              1 :
              descriptor  =  remove_plane
              plane_tol  =  0.005
              2 :
              descriptor  =  dbscan
              min_samples  =  10
              search_radius  =  0.01
              3 :
              cluster_size  =  [40, 10000000]
              descriptor  =  filter_by_size
              max_clouds_returned  =  100
              x_axis  =  [0.008, 0.2]
              y_axis  =  [0.008, 0.1]
              z_axis  =  [0.006, 0.1]
              4 :
              descriptor  =  sort_clouds_height
              high_to_low  =  True
              5 :
              descriptor  =  find_grasp
              rotate_z  =  True
       view_distance  =  0.15


In this example, the feature is found by performing a downsampling step, removing the table plane, a segmentation step, a filter based on object size (<code>x_axis, y_axis, z_axis</code>), then sorts objects by their height (<code>high_to_low = True</code>).

# Select Feature

In [6]:
#Select a feature from the feature library
your_feature = 'default'

# Define Waypoints

<i><center><h3>Warning: make sure the robot can move safely between the start_point and the drop_point</h3><i></center>
    
We will now define a couple of way points that will serve the robot as starting and drop-off locations. More complex environments might require the robot to sweep the workspace or generate a drop-off location based on what is already in a container, e.g., for palletizing. 


## Start Point

In [7]:
# 1. Move the smarthand to the view distance specified by your feature as shown in the picture above. The default feature uses a view distance of 0.15m 

In [8]:
# 2. Align Gripper, use this function to align the gripper to the nearest axis
rm.align_gripper_with_axis()

True

In [9]:
# 3. Save waypoint
start_point = rm.get_tcp_pose()

## Drop Point

In [10]:
# 1. # Move the smarthand to your desired object drop point

In [11]:
# 2. Align Gripper, use this function to align the gripper to the nearest axis
rm.align_gripper_with_axis()

True

In [12]:
# 3. Save waypoint
drop_point = rm.get_tcp_pose()

# Find Feature

To find a feature and a potential grasp, the robot will first move to `start_point` and open the gripper. (If the gripper is closed, the robot will not have its entire field of view at its disposal and might miss part of the workspace.) The `find_feature` function allows you to specify, which of the steps in the `capture_process_list` (Section <i>Select Feature</i>) should be displayed to the user. In this example, we selected `c` to view the raw capture, `2` to view the dbscan segmented cloud, and `5` to view the grasp found.
 
 

In [14]:
# Move to start point
rm.movej(start_point)

True

In [15]:
# Open gripper
rm.open_gripper()

True

In [16]:
# Run find_feature to get a capture and run the process list
grasps = rm.find_feature(lib[your_feature], output=['c', '2', '5'])

Capture Cloud Output:


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.00534798205755243, 0.4592661888557138, 0.…

Segment Cloud DBScan Output:


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.0007770934366057768, 0.2224529075699343, …

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.00491719346796557, 0.44811141964169116, 0…

# Grasps

A `grasp` is a datastructure consisting of a robot pose and a grasp width. The grasp generator returns a list of grasps that are sorted based on the specific grasp generator that has been selected in the `capture_process_list`. 

In [17]:
# The returned grasp is a nested list of grasps, by selecting index 0 we can view the first grasp returned.
grasp = grasps[0]
# A grasp consists of a pose (4x4 homogeneous transformation matrix) and a grasp width (float)
print('Pose:')
print(grasp[0])
print()
print('Grasp Width:')
print(grasp[1])

Pose:
[[-9.98982078e-01 -4.51088361e-02 -1.77614306e-05 -1.34612622e-01]
 [-4.51088007e-02  9.98980622e-01  1.70678207e-03 -3.97567255e-01]
 [-5.92476277e-05  1.70584590e-03 -9.99998543e-01  2.77650075e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Grasp Width:
0.02499535776717239


# Pick Object

In order to safely grasp an object, we first will need to move to a safe position just above the grasp pose. We then set the gripper width to the size returned by the grasp generator, and finally move to the grasp pose, close the gripper and return to the drop-off location after revisiting the safe pre-grasp pose.

In [18]:
#Move to grasp pose approach
rm.move_tcp_over_pose(grasp[0], distance=0.15, rotate=True)

In [19]:
# Set gripper width
rm.set_gripper_width(grasp[1])

True

In [20]:
# Move to pose
rm.movel(grasp[0])

True

In [21]:
# Close gripper
rm.close_gripper()

True

In [22]:
# Move back to grasp pose approach
rm.move_tcp_over_pose(grasp[0], distance=0.15)

In [23]:
# Move to drop point
rm.movej(drop_point)

True

In [24]:
# Open gripper
rm.open_gripper()

True